In [1]:
import random
import scispacy
import spacy
from spacy.util import minibatch, compounding
from spacy.language import Language
import os

In [2]:
par_dir = os.path.abspath(os.path.join(os.getcwd(), os.pardir))

In [3]:
par_dir

'/Users/tylerursuy/Desktop/MSDS/603/product-analytics-group-project-armor'

In [5]:
bc5cdr_md = spacy.load("{}/models/en_ner_bc5cdr_md-0.1.0".format(par_dir))

### Abnormal Echocardiogram Test Case

In [6]:
disease_example = "Abnormal echocardiogram findings and followup. \
Shortness of breath, congestive heart failure, and valvular insufficiency."

In [7]:
drug_example = "Aspirin 325 mg once a day. Metoprolol 50 mg once a day, \
but we have had to hold it because of relative bradycardia which he apparently has a history of. \
Nexium 40 mg once a day. Zocor 40 mg once a day, and there is a fasting \
lipid profile pending at the time of this dictation. I see that \
his LDL was 136 on May 3, 2002. Plavix 600 mg p.o. x1 which I am giving him tonight"


#### bc5cdr_md NER

In [8]:
for e in bc5cdr_md(disease_example).ents:
    print(f"Entity: {e}, Type: {e.label_}")

Entity: Shortness of breath, Type: DISEASE
Entity: congestive heart failure, Type: DISEASE
Entity: valvular insufficiency, Type: DISEASE


In [9]:
for e in bc5cdr_md(drug_example).ents:
    print(f"Entity: {e}, Type: {e.label_}")

Entity: Aspirin, Type: CHEMICAL
Entity: Metoprolol, Type: CHEMICAL
Entity: bradycardia, Type: DISEASE
Entity: p.o, Type: CHEMICAL


### Fixing Errors

In [10]:
TRAIN_DATA = [
    ("Nexium 40 mg once a day.", {"entities": [(0, 6, "CHEMICAL")]}),
    ("Zocor 40 mg once a day, and there is a fasting lipid profile pending at the time of this dictation. I see that his LDL was 136 on May 3, 2002.", 
     {"entities": [(0, 5, "CHEMICAL")]}),
    ("Plavix 600 mg p.o. x1 which I am giving him tonight.", {"entities": [(0, 6, "CHEMICAL")]}),
    ("Abnormal echocardiogram findings and followup,", {"entities": [(0, 23, "DISEASE")]})
]

In [13]:
def train(model=None, output_dir=None, n_iter=100):
    """Load the model, set up the pipeline and train the entity recognizer."""
    if model is not None:
        nlp = spacy.load(model)  # load existing spaCy model
        print("Loaded model '%s'" % model)
    else:
        nlp = spacy.blank("en")  # create blank Language class
        print("Created blank 'en' model")

    # create the built-in pipeline components and add them to the pipeline
    # nlp.create_pipe works for built-ins that are registered with spaCy
    if "ner" not in nlp.pipe_names:
        ner = nlp.create_pipe("ner")
        nlp.add_pipe(ner, last=True)
    # otherwise, get it so we can add labels
    else:
        ner = nlp.get_pipe("ner")

    # add labels
    for _, annotations in TRAIN_DATA:
        for ent in annotations.get("entities"):
            ner.add_label(ent[2])

    # get names of other pipes to disable them during training
    other_pipes = [pipe for pipe in nlp.pipe_names if pipe != "ner"]
    with nlp.disable_pipes(*other_pipes):  # only train NER
        # reset and initialize the weights randomly – but only if we're
        # training a new model
        if model is None:
            nlp.begin_training()
        for itn in range(n_iter):
            random.shuffle(TRAIN_DATA)
            losses = {}
            # batch up the examples using spaCy's minibatch
            batches = minibatch(TRAIN_DATA, size=compounding(4.0, 32.0, 1.001))
            for batch in batches:
                texts, annotations = zip(*batch)
                nlp.update(
                    texts,  # batch of texts
                    annotations,  # batch of annotations
                    drop=0.5,  # dropout - make it harder to memorise data
                    losses=losses,
                )
            print("Losses", losses)

    # test the trained model
    for text, _ in TRAIN_DATA:
        doc = nlp(text)
        print("Entities", [(ent.text, ent.label_) for ent in doc.ents])
    return nlp

In [14]:
model_dir = "{}/models/en_ner_bc5cdr_md-0.1.0".format(par_dir)
bc5cdr_md = train(model_dir, n_iter=5)

Loaded model '/Users/tylerursuy/Desktop/MSDS/603/product-analytics-group-project-armor/models/en_ner_bc5cdr_md-0.1.0'
Losses {'ner': 0.5698283570474955}
Losses {'ner': 0.7930012403959381}
Losses {'ner': 0.39752292080234397}
Losses {'ner': 0.37710215020917204}
Losses {'ner': 0.1763788676717013}
Entities []
Entities [('Zocor', 'CHEMICAL')]
Entities [('Plavix', 'CHEMICAL')]
Entities [('Nexium', 'CHEMICAL')]


In [25]:
for e in bc5cdr_md(drug_example).ents:
    print(f"Entity: {e}, Type: {e.label_}")

Entity: Aspirin, Type: CHEMICAL
Entity: Metoprolol, Type: CHEMICAL
Entity: bradycardia, Type: DISEASE
Entity: Nexium, Type: CHEMICAL
Entity: Zocor, Type: CHEMICAL


In [26]:
for e in bc5cdr_md(disease_example).ents:
    print(f"Entity: {e}, Type: {e.label_}")

Entity: Abnormal echocardiogram, Type: DISEASE
Entity: Shortness of breath, Type: DISEASE
Entity: congestive heart failure, Type: DISEASE
Entity: valvular insufficiency, Type: DISEASE
